In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# from deepspeed import comm as dist

ImportError: cannot import name 'comm' from 'deepspeed' (/nas2/kf/miniconda3/envs/pytorch1.10/lib/python3.8/site-packages/deepspeed/__init__.py)

In [4]:
from deepspeed.pipe import PipelineModule, LayerSpec, TiedLayerSpec

In [5]:
import sys
import os

In [6]:
import torch.nn as nn

In [11]:
sys.path.append('/nas/shawn_guo/gpt_neox/')

In [27]:
from gpt2_model_gptj import GPT2ModelPipe

In [14]:
import deepspeed

In [1]:
from deepspeed import init_distributed

In [2]:
help(init_distributed)

Help on function init_distributed in module deepspeed.utils.distributed:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None)
    Initialize torch.distributed backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
    
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
    
        distributed_port: Optional (int). torch distributed backend port
    
        verbose: Optional (bool). verbose logging
    
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
    
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is s

In [3]:
os.environ["MASTER_ADDR"] = '192.168.53.6'
os.environ["MASTER_PORT"] = '8920'
os.environ["WORLD_SIZE"] = '2'

NameError: name 'os' is not defined

In [ ]:
# from deepspeed.comm.comm import init_distributed
# init_distributed(dist_backend="nccl",
#                      auto_mpi_discovery=True,
#                     distributed_port=TORCH_DISTRIBUTED_DEFAULT_PORT,
#                     verbose=True,
#                     timeout=default_pg_timeout,
#                      init_method=None,
#                     dist_init_required=None,
#                      config=None):

In [22]:
from dataclasses import dataclass

In [23]:
@dataclass
class GPTJArgs:
    hidden_size = 4096
    checkpoint_num_layers = 1
    num_layers = 28
    attention_config = ['global']*28
    checkpoint_activations = False
    pipe_partition_method = 'parameters'  # default
    no_weight_tying = False
    padded_vocab_size = 50400
    max_position_embeddings = 2048
    hidden_dropout = 0.0 
    pos_emb = "rotary" # gxh
    num_attention_heads = 16
    fp16_lm_cross_entropy = False # gxh
    is_pipe_parallel = True
    init_method = "normal"
    output_layer_init_method = "normal"
    init_method_std = "normal"
    norm = "layernorm"
    layernorm_epsilon = 1e-4
    deepspeed = True
    use_bnb_optimizer = False
    opt_pos_emb_offset = 0
    n_embd = 4096
    layer_norm_epsilon = 1e-5
    n_inner = None
    attn_pdrop = 0.0
    attn_pdrop = 0.1
    resid_pdrop = 0.0
    rotary_dim = 64
    activation_function = 'gelu_new'
    num_stages = 8
    scaled_upper_triang_masked_softmax_fusion = False

In [25]:
gptj_args = GPTJArgs()

In [18]:
from megatron.utils import print_rank_0

/nas2/kf/miniconda3/envs/pytorch1.10/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
fatal: detected dubious ownership in repository at '/nas/shawn_guo/gpt_neox'
To add an exception for this directory, call:

	git config --global --add safe.directory /nas/shawn_guo/gpt_neox


In [19]:
def get_model(gptj_args, use_cache=False):
    """Build the model."""

    print_rank_0("building GPT2 model ...")

    # Build model on cpu.
    model = GPT2ModelPipe(
        gptj_args=gptj_args,
        num_tokentypes=0,
        parallel_output=True,
        topology=X,
        use_cache=use_cache,
    )

    if gptj_args.is_pipe_parallel:
        # Export PipeParallel model to nn.Sequential model to avoid the overhead of deepspeed's pipe parallel training
        model = model.to_sequential()

    if gptj_args.deepspeed:
        # DeepSpeed handles CUDA, FP16, and DDP components.
        return model
    else:
        raise ValueError("Must be using deepspeed to run neox")

In [26]:
model = get_model(gptj_args)

building GPT2 model ...


NameError: name 'GPT2ModelPipe' is not defined

In [2]:
import deepspeed

/nas/shawn_guo/miniconda3/envs/torch1.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from deepspeed import comm as dist

In [4]:
help(deepspeed.init_distributed)

Help on function init_distributed in module deepspeed.comm.comm:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None, dist_init_required=None, config=None)
    Initialize dist backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
        distributed_port: Optional (int). torch distributed backend port
        verbose: Optional (bool). verbose logging
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is specified

In [14]:
help(deepspeed.init_distributed)

Help on function init_distributed in module deepspeed.comm.comm:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None, dist_init_required=None, config=None)
    Initialize dist backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
        distributed_port: Optional (int). torch distributed backend port
        verbose: Optional (bool). verbose logging
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is specified

In [16]:
import torch

In [19]:
help(deepspeed.init_distributed)

Help on function init_distributed in module deepspeed.comm.comm:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None, dist_init_required=None, config=None)
    Initialize dist backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
        distributed_port: Optional (int). torch distributed backend port
        verbose: Optional (bool). verbose logging
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is specified

In [5]:
import os

In [6]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

In [42]:
dist.init_deepspeed_backend(ds_backend='nccl')

[2022-09-09 16:35:58,837] [WARNING] [comm.py:160:init_deepspeed_backend] NCCL backend in DeepSpeed not yet implemented


In [44]:
dist.init_distributed(dist_backend='nccl',auto_mpi_discovery=False,init_method='env://')

[2022-09-09 16:37:54,222] [INFO] [comm.py:635:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set

In [46]:
import torch

In [ ]:
torch.distributed.init_process_group(backend="nccl", init_method="env://", world_size=torch.cuda.device_count(),
                                              rank=0)

In [30]:
help(dist.init_distributed())

Help on function init_distributed in module deepspeed.comm.comm:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None, dist_init_required=None, config=None)
    Initialize dist backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
        distributed_port: Optional (int). torch distributed backend port
        verbose: Optional (bool). verbose logging
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is specified

In [112]:
model

SequentialWrapper(
  (sequential): Sequential(
    (0): EmbeddingPipe(
      (word_embeddings): Embedding(50400, 4096)
      (embedding_dropout): Dropout(p=0.0, inplace=False)
    )
    (1): Lambda()
    (2): TransformerLayerPipe(
      (ln_1): LayerNorm((4096,), eps=0.0001, elementwise_affine=True)
      (attn): GPTJAttention(
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): GPTJMLP(
        (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
        (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
        (act): GELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
     

In [113]:
from transformers import GPTJConfig

In [19]:
import deepspeed

In [24]:
from deepspeed.runtime.pipe.topology import PipelineParallelGrid

In [20]:
from deepspeed import comm as dist

In [1]:
import torch

/nas/shawn_guo/miniconda3/envs/torch1.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
torch.distributed.init_process_group(backend="nccl",
                        init_method="file:///distributed_test",
                        world_size=2,
                        rank=0)

RuntimeError: open(/distributed_test): Permission denied

In [24]:
dist.init_deepspeed_backend(ds_backend='nccl')

[2022-09-09 15:28:55,553] [WARNING] [comm.py:160:init_deepspeed_backend] NCCL backend in DeepSpeed not yet implemented


In [28]:
help(dist.init_distributed)

Help on function init_distributed in module deepspeed.comm.comm:

init_distributed(dist_backend='nccl', auto_mpi_discovery=True, distributed_port=29500, verbose=True, timeout=datetime.timedelta(seconds=1800), init_method=None, dist_init_required=None, config=None)
    Initialize dist backend, potentially performing MPI discovery if needed
    
    Arguments:
        dist_backend: Optional (str). torch distributed backend, e.g., nccl, mpi, gloo
        auto_mpi_discovery Optional (bool). if distributed environment variables are not set, attempt to discover them from MPI
        distributed_port: Optional (int). torch distributed backend port
        verbose: Optional (bool). verbose logging
        timeout: Optional (timedelta). Timeout for operations executed against the process group. Default value equals 30 minutes.
        init_method: Optional (string). Torch distributed, URL specifying how to initialize the process group. Default is “env://” if no init_method or store is specified

In [25]:
if dist.init_distributed():
    print('yes')

[2022-09-08 12:55:50,832] [INFO] [comm.py:618:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2022-09-08 12:55:50,950] [INFO] [comm.py:675:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.53.6, master_port=29500
[2022-09-08 12:55:50,952] [INFO] [comm.py:635:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [27]:
dist.get_world_size()

1

In [15]:
from deepspeed.runtime.pipe import ProcessTopology

In [87]:
dist.get_world_size()

1

In [45]:
net = nn.Sequential(
    nn.Linear(256, 512),
    nn.ReLU(inplace=True),
    nn.Linear(512, 2)
)
from deepspeed.pipe import PipelineModule
net = PipelineModule(layers=net, num_stages=2)

SEED_LAYERS=False BASE_SEED=1234 SEED_FN=None


RuntimeError: num_stages (2) must divide distributed world size (1)

In [1]:
import torch
torch.distributed.get_world_size()

/nas/shawn_guo/miniconda3/envs/torch1.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
torch.distributed.init_process_group(backend="nccl")

In [28]:
import deepspeed

In [52]:
deepspeed_config = 'deepspeed_config.json'

In [53]:
args.deepspeed_config = deepspeed_config

In [ ]:
args

In [55]:
model_engine, optimizer, _, _ = deepspeed.initialize(args,
                                                     model=net,
                                                     model_parameters=net.parameters())

[2022-09-09 09:30:08,651] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.7.2, git-hash=unknown, git-branch=unknown
[2022-09-09 09:30:08,653] [WARNING] [config.py:55:read_zero_config_deprecated] DeepSpeedConfig: this format of ZeRO optimization setup is deprecated. Please use the following format: 
ZeRO optimization should be enabled as:
"session_params": {
  "zero_optimization": {
    "stage": [0|1|2],
    "stage3_max_live_parameters" : 1000000000,
    "stage3_max_reuse_distance" : 1000000000,
    "allgather_partitions": [true|false],
    "allgather_bucket_size": 500000000,
    "reduce_scatter": [true|false],
    "contiguous_gradients" : [true|false]
    "overlap_comm": [true|false],
    "reduce_bucket_size": 500000000,
    "load_from_fp32_weights": [true|false],
    "cpu_offload": [true|false] (deprecated),
    "cpu_offload_params" : [true|false] (deprecated),
    "cpu_offload_use_pin_memory": [true|false] (deprecated),
    "sub_group_size" : 1000000000000,
    "of

PermissionError: [Errno 13] Permission denied: '/tmp/torch_extensions/fused_adam'

In [39]:
deepspeed_config = NeoXArgsDeepspeedConfig()

In [35]:
class NeoXArgsDeepspeedConfig:
    """
    Args for deepspeed config
    Every argument included here will be included in deepspeed config json
    #TODO this list is not complete as compared to https://www.deepspeed.ai/docs/config-json/
    """

    deepspeed: bool = True
    """boolean flag to enable DeepSpeed (Always True)"""

    train_batch_size: int = None
    """
    The effective training batch size. This is the amount of data samples that leads to one step of model update. train_batch_size is aggregated by the batch size that a single GPU processes in one forward/backward pass (a.k.a., train_step_batch_size), the gradient accumulation steps (a.k.a., gradient_accumulation_steps), and the number of GPUs.
    """

    train_micro_batch_size_per_gpu: int = None
    """
    Batch size to be processed by one GPU in one step (without gradient accumulation). When specified, gradient_accumulation_steps is automatically calculated using train_batch_size and number of GPUs. Should not be concurrently specified with gradient_accumulation_steps in the configuration JSON.
    """

    gradient_accumulation_steps: int = 1
    """
    Number of training steps to accumulate gradients before averaging and applying them. This feature is sometimes useful to improve scalability since it results in less frequent communication of gradients between steps. Another impact of this feature is the ability to train with larger batch sizes per GPU. When specified, train_step_batch_size is automatically calculated using train_batch_size and number of GPUs. Should not be concurrently specified with train_step_batch_size in the configuration JSON.
    """

    optimizer: dict = None
    """
    dict containing the keys type and params

    type: The optimizer name. DeepSpeed natively supports Adam, AdamW, OneBitAdam, Lamb, and OneBitLamb optimizers (See here for details) and will import other optimizers from torch.

    params: Dictionary of parameters to instantiate optimizer. The parameter names must match the optimizer constructor signature (e.g., for Adam).
    """

    scheduler: dict = None
    """
    dict containing the keys type and params

    type: The scheduler name. See here (https://deepspeed.readthedocs.io/en/latest/schedulers.html) for list of support schedulers.

    params: Dictionary of parameters to instantiate scheduler. The parameter names should match scheduler constructor signature.
    """

    fp32_allreduce: bool = False
    """
    During gradient averaging perform allreduce with 32 bit values
    """

    prescale_gradients: bool = False
    """
    Scale gradients before doing allreduce
    """

    gradient_predivide_factor: float = 1.0
    """
    Before gradient averaging predivide gradients by a specified factor, can sometimes help with fp16 stability when scaling to large numbers of GPUs
    """

    sparse_gradients: bool = False
    """
    Enable sparse compression of torch.nn.Embedding gradients.
    """

    fp16: dict = None
    """
    Configuration for using mixed precision/FP16 training that leverages NVIDIA’s Apex package.
    """

    amp: dict = None
    """
    Dictionary as described in Deepspeed documentation: https://www.deepspeed.ai/docs/config-json/#automatic-mixed-precision-amp-training-options
    """

    gradient_clipping: float = 0.0
    """
    Enable gradient clipping with provided value
    """

    zero_optimization: dict = None
    """"""

    steps_per_print: int = 10
    """
    Print train loss every N steps.
    """

    wall_clock_breakdown: bool = False
    """
    Enable timing of the latency of forward/backward/update training phases.
    """

    dump_state: bool = False
    """
    Print out state information of DeepSpeed object after initialization.
    """

    flops_profiler: dict = None
    """
    Dictionary as described in Deepspeed documentation: https://www.deepspeed.ai/docs/config-json/#flops-profiler
    """

    zero_allow_untested_optimizer: bool = False
    """
    Whether Deepspeed Zero Optimizer will allow an optimizer that hasn't been tested by the deepspeed team
    """